## PyTorch Tutorial: CIFAR-10 classification

Source: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-an-image-classifie

In [1]:
import argparse
import math
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import datasets, transforms


In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
# Args

# data loader
batch_size = 4
test_batch_size = 4

# trainer
epochs = 2

# optimizer
lr = 0.001
momentum = 0.9
# no_cuda = True  # not used
seed = 1
log_interval = 2000

In [4]:
torch.manual_seed(seed)

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

<Figure size 640x480 with 1 Axes>

plane   dog  deer  deer


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [8]:
nn.CrossEntropyLoss?

In [9]:
criterion = nn.CrossEntropyLoss(reduce=False)
sgd_optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [10]:
def train(model, data_loader, optimizer, criterion, epoch, log_interval):
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels).sum()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % log_interval == (log_interval - 1):
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                (batch_idx + 1) * len(inputs),
                len(data_loader.dataset),
                100. * (batch_idx + 1) / len(data_loader),
                running_loss / log_interval))
            running_loss = 0.0
    return


In [12]:
def test(model, data_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            losses = criterion(outputs, labels)
            test_loss += torch.sum(losses)
            pred = outputs.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()
    n_cases = len(data_loader.dataset)
    print("n_cases:", n_cases)
    test_loss /= n_cases
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,
        correct,
        len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    return


In [13]:
# test(net, test_loader, criterion)
for epoch in range(1, epochs + 1):
    train(net, train_loader, sgd_optimizer, criterion, epoch, log_interval)
    test(net, test_loader, criterion)


Train Epoch: 1 [08000/50000 (16%)]	Loss: 8.050952
Train Epoch: 1 [16000/50000 (32%)]	Loss: 6.900259
Train Epoch: 1 [24000/50000 (48%)]	Loss: 6.661740
Train Epoch: 1 [32000/50000 (64%)]	Loss: 6.423126
Train Epoch: 1 [40000/50000 (80%)]	Loss: 6.331477
Train Epoch: 1 [48000/50000 (96%)]	Loss: 6.200371
n_cases: 10000

Test set: Average loss: 1.5439, Accuracy: 4440/10000 (44%)

Train Epoch: 2 [08000/50000 (16%)]	Loss: 6.081241
Train Epoch: 2 [16000/50000 (32%)]	Loss: 6.019436
Train Epoch: 2 [24000/50000 (48%)]	Loss: 5.985378
Train Epoch: 2 [32000/50000 (64%)]	Loss: 5.912212
Train Epoch: 2 [40000/50000 (80%)]	Loss: 6.033348
Train Epoch: 2 [48000/50000 (96%)]	Loss: 5.985319
n_cases: 10000

Test set: Average loss: 1.4600, Accuracy: 4784/10000 (48%)



In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    

print('Finished Training')

In [ ]:
print('''
Original:

[1,  2000] loss: 2.186
[1,  4000] loss: 1.841
[1,  6000] loss: 1.673
[1,  8000] loss: 1.563
[1, 10000] loss: 1.511
[1, 12000] loss: 1.440
[2,  2000] loss: 1.399
[2,  4000] loss: 1.353
[2,  6000] loss: 1.329
[2,  8000] loss: 1.300
[2, 10000] loss: 1.292
[2, 12000] loss: 1.289
Finished Training''')

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))